In [ ]:
#pip install openai

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Union
import seaborn as sns
import datetime
import requests
from bs4 import BeautifulSoup

# 158739 Assignment 4 - Does having wealthy parents make you a better tennis player?

#### Student name: James Evans and Hayley Wikeepa
#### Student ID: 12211872


# Introduction



Does having wealth parent make you a better tennis player? There are lots of ways to analise this. We can look at if the current world top tennis players are mostly from wealth families. We can look if being from a wealthy family makes it more likely you will win when playing profesional tennis. Assuming this is true we can look if having wealthy parents is "priced in" to the betting odds when betting on tennis matches.

Two things that make us wonder about tennis and wealthy parents. Firstly there are two of the top 100 woman tennis players have billionare parent. Second a tweet one of us saw wondering why so many formula one drivers fathers have hyperlinks in wikipedia (hinting that you need a rich father to become a formular 1 driver).





#### Datasets used:
- ATP (Mens profesional tennis assossiation) and WTA (Womens profesional tennis assossiation) tennis player lists
- Tennis results from the past ?? years
- Tennis betting odds from the last ?? years
- List of tennis players that have wiki pages, and a boolean if there parents have wiki pages
- chat gpt answers to the question "did this tennis player have wealthy parents?"


#### Dataset sources: 

- 
- 


### Research Questions

1. Is having wealth parents a predictor of becoming a profectional tennis player?
2. Is having wealth parents a predictor of future sucssess as a profectional tennis player?
3. Is there money to be made by taking into account if a tennis player has wealth parents, when betting on a tennis mathches?


### Executive Summary



## What do we need to achive?


### Ideas of what we could find out

First we could see if there is a disporportinate amount of pro tennis players that have wealth parents (I am sure there will be).

Would could see if there is a correlation between rich partents and tennis success when playing as a pro (look at results from pro matches). 
we could look at win % vs wealthy parents

If answer is yes, wealth parents make you play better as a pro, then we can look to see if this is taken into account in the betting odds.
For each player, we could work out how much you would have won / lost, if you had bet a constant amount on that player, over there last x number of games. We can then have a way of saying who is a profitable player and who is an unprofitable player to bet on. We can then see if this correlates with if they have rich/famous parents.


### Marking info

- Data Acquisition -     20
    * Diversity of sources (at least one must be dynamic – full marks for using both APIs and web scraping – penalties will be applied for re-using examples from class)
    * Appropriate use of merging and concatenation.
    
- Data Wrangling and EDA  -  30
- Data Analysis -  35
- Originality and challenge 15

BONUS
- Big Data ProcessingTechniques - 5

### Other notes from the brief
Clearly introduce your problem domain, articulate your research questions and provide an executive summary at the beginning. Follow the provided Jupyter notebook template.
You must document and explain the reasoning behind the coding steps you are taking and provide explanations of all your graphs and tables as is appropriate. Make sure you label all aspects of your graphs.
The activities listed under the five stages in the workflow diagram above are a guide only. This means that operations like group-by statements as well as pivot tables could be a part of the ‘Data Wrangling’ phase as EDA, and not only a part of the data analysis phase. Finally, please run your report through an external spell checker and feel free to use ChatGPT judiciously to help you as discussed in class.



In [2]:
# headers for API

headers = {
	"X-RapidAPI-Key": "8556d4b2f5mshddae5c2b7778158p1b7b83jsn131f0acf695b",
	"X-RapidAPI-Host": "ultimate-tennis1.p.rapidapi.com"
}

### All API calls

#### Get all ATP tournament results

In [3]:
# Get all ATP results from 2019 - 2022 for all tournaments into one dataframe

base_url = "https://ultimate-tennis1.p.rapidapi.com/tournament_list/atp/{year}/atpgs"

# get all ATP tounament results
ATP_tornaments_list = []

for year in range(datetime.datetime.now().year - 4, datetime.datetime.now().year):
    url = base_url.format(year=year)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['Tournaments'])
        ATP_tornaments_list.append(df)
    else:
        print(f"Error retrieving data for year {year}. Status code: {response.status_code}")

#### Get all WTA tournament results

In [30]:
# Get all WTA results from 2019 - 2022 for all tournaments into one dataframe

base_url = "https://ultimate-tennis1.p.rapidapi.com/tournament_list/wta/{year}"

# get all ATP tounament results
WTA_tornaments_list = []

for year in range(datetime.datetime.now().year - 4, datetime.datetime.now().year):
    url = base_url.format(year=year)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['Tournaments'])
        WTA_tornaments_list.append(df)
    else:
        print(f"Error retrieving data for year {year}. Status code: {response.status_code}")


#### All ATP player details and statistics

In [7]:
# All ATP player details and statistics

url = "https://ultimate-tennis1.p.rapidapi.com/global_players_stats/aces/all/all"

ATP_player_stats_response = requests.get(url, headers=headers)

#### All WTA player details and statistics

In [8]:
# all WTA player details

url = "https://ultimate-tennis1.p.rapidapi.com/rankings/wta/singles/200/current"

wta_rank_response = requests.get(url, headers=headers)

### DataFrame edits

#### ATP tournament results to DataFrame

In [119]:
# combine all dfs in ATP_tournaments list        
ATP_tornaments = pd.concat(ATP_tornaments_list)
ATP_tornaments = ATP_tornaments.reset_index(drop=True)       

# normalise winners column
winners_df = pd.json_normalize(ATP_tornaments['Winners'])
temp_ATP_df = pd.concat([ATP_tornaments, winners_df], axis=1)
ATP_tornaments = temp_ATP_df.drop('Winners', axis=1)

# normalise surfaces column to match wta tournaments df
surfaces_df = ATP_tornaments['Surface'].str.split(' ', expand=True)
surfaces_df.columns = ['Outdoor/Indoor', 'surface']
temp_df = pd.concat([ATP_tornaments, surfaces_df], axis=1)
ATP_tornaments = temp_df.drop('Surface', axis=1)

ATP_tornaments['year'] = pd.to_datetime(ATP_tornaments['Timestamp']).dt.year
ATP_tornaments = ATP_tornaments.drop(['Timestamp', 'doubles'], axis=1)
ATP_tornaments.rename(columns={'Total Prize Money': 'prizeMoney'}, inplace=True)

In [120]:
# create unique index for tornaments - ID was not unique
ATP_tornaments['index'] = ATP_tornaments['ID'].astype(str) + '_' + ATP_tornaments['Location'].astype(str)
ATP_tornaments.set_index('index', inplace=True)

#### WTA tournament results to DataFrame

In [121]:
# combine all dfs in WTA_tournaments list        
wta_tournaments = pd.concat(WTA_tornaments_list)
wta_tournaments = wta_tournaments.reset_index(drop=True)       

In [122]:
wta_tournaments['prizeMoney'] = wta_tournaments['prizeMoney'].apply(lambda x: '${:,.2f}'.format(x))
fullnames_df = wta_tournaments['Full Name'].str.split(' - ', expand=True)
fullnames_df.columns = ['Tournament Name', 'Location']
temp_df = pd.concat([wta_tournaments, fullnames_df], axis=1)

# drop unused columns
wta_tournaments = wta_tournaments.drop(['endDate', 'startDate', 'category', 'city', 'name',
                                        'country', 'doublesDrawSize', 'prizeMoneyCurrency', 'singlesDrawSize'], axis=1)

# normalise Full Names column
fullnames_df = wta_tournaments['Full Name'].str.split(' - ', expand=True)
fullnames_df.columns = ['Tournament Name', 'Location']
temp_df = pd.concat([wta_tournaments, fullnames_df], axis=1)
wta_tournaments = temp_df.drop(['Full Name'], axis=1)

In [123]:
# create unique index for tornaments - ID was not unique
wta_tournaments['index'] = wta_tournaments['ID'].astype(str) + '_' + wta_tournaments['Location'].astype(str)
wta_tournaments.set_index('index', inplace=True)

#### join tournaments into one dataframe

In [141]:
all_tournaments = pd.concat([ATP_tornaments, wta_tournaments], ignore_index=True)
all_tournaments.head()

,ID,Location,prizeMoney,Tournament Name,singles,Outdoor/Indoor,surface,year
0,451,"Doha, Qatar","$1,416,205",Doha,bd06,Outdoor,Hard,2018
1,339,"Brisbane, Australia","$589,680",Brisbane,n552,Outdoor,Hard,2018
2,891,"Pune, India","$589,680",Pune,a678,Outdoor,Hard,2018
3,338,"Sydney, Australia","$589,680",Sydney,dh58,Outdoor,Hard,2019
4,301,"Auckland, New Zealand","$589,680",Auckland,sl08,Outdoor,Hard,2019


#### Get  top 200 ATP and WTA players into a single DataFrame

In [130]:
# edit ATP dataframe to show basic player details

ATP_player_stats = pd.DataFrame(ATP_player_stats_response.json()['data'])
ATP_player_stats.set_index('ID', inplace=True)

ATP_player_stats = ATP_player_stats.drop(['Aces', 'Games played'], axis=1)


In [132]:
# edit WTA players dataframe to show basic player details

wta_player_stats = pd.DataFrame(wta_rank_response.json()['data'])
wta_player_stats.set_index('ID', inplace=True)

wta_player_stats = wta_player_stats.drop(['movement', 'rankedAt', 'country', 'points', 'tournamentsPlayed'], axis=1)
wta_player_stats.rename(columns={'ranking': 'Rank'}, inplace=True)
wta_player_stats.rename(columns={'name': 'Name'}, inplace=True)


In [133]:
top_players = pd.concat([ATP_player_stats, wta_player_stats])

In [144]:
# top_players.head()

### all tournaments with only ATP results and NaN WTA results

In [143]:
all_tournament_results = all_tournaments.merge(top_players, left_on='singles', right_on='ID')
all_tournament_results.to_csv('all_tornaments.csv')

### read csv files
notes on what each column means can be found in ../datasets/datasets_notes.txt

In [29]:
wta2019_tornaments = pd.read_csv('datasets/2019wta.csv')
wta2020_tornaments = pd.read_csv('datasets/2020wta.csv')
wta2021_tornaments = pd.read_csv('datasets/2021wta.csv')
wta2022_tornaments = pd.read_csv('datasets/2022wta.csv')
wta2023_tornaments = pd.read_csv('datasets/2023wta.csv')

In [34]:
atp2019_tornaments = pd.read_csv('datasets/2019atp.csv')
atp2020_tornaments = pd.read_csv('datasets/2020atp.csv')
atp2021_tornaments = pd.read_csv('datasets/2021atp.csv')
atp2022_tornaments = pd.read_csv('datasets/2022atp.csv')
atp2023_tornaments = pd.read_csv('datasets/2023atp.csv')

In [35]:
atp_dfs = [atp2019_tornaments, atp2020_tornaments, atp2021_tornaments, atp2022_tornaments, atp2023_tornaments]
atp_tournaments_wOdds = pd.concat(atp_dfs)

#### edit tournaments with odds DataFrames

In [102]:
# join wta & atp tournament with betting odds DataFrames
all_tournaments_wOdds = pd.concat([wta_tournaments_wOdds, atp_tournaments_wOdds])

# only get finals from each tournament
all_tournaments_wOdds = all_tournaments_wOdds.loc[all_tournaments_wOdds['Round'] == 'The Final']

In [103]:
all_tournaments_wOdds['year'] = pd.to_datetime(all_tournaments_wOdds['Date'], format='%d/%m/%Y').dt.year

all_tournaments_wOdds = all_tournaments_wOdds.drop(
    ['Round', 'Tier', 'Comment', 'Loser', 'WPts', 'LPts', 'Wsets', 'Lsets', 'B365L', 'PSL', 'MaxL', 'AvgL',
     'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Series', 'Best of', 'Date'], axis=1)

all_tournaments_wOdds['WTA'] = all_tournaments_wOdds['WTA'].fillna(0)
all_tournaments_wOdds['ATP'] = all_tournaments_wOdds['ATP'].fillna(0)


In [104]:
# create unique index for all tournamnets with odds
all_tournaments_wOdds['index'] = all_tournaments_wOdds['WTA'].astype(str) + '_' + all_tournaments_wOdds['ATP'].astype(str) + '_' + all_tournaments_wOdds['Tournament'].astype(str)
all_tournaments_wOdds.set_index('index', inplace=True)

In [105]:
all_tournaments_wOdds.head()

,WTA,Location,Tournament,Court,Surface,Winner,WRank,LRank,B365W,PSW,MaxW,AvgW,ATP,year
index,,,,,,,,,,,,,,
1.0_0.0_ASB Classic,1.0,Auckland,ASB Classic,Outdoor,Hard,Goerges J.,14.0,152.0,1.40,1.43,1.50,1.42,0.0,2019
2.0_0.0_Brisbane International,2.0,Brisbane,Brisbane International,Outdoor,Hard,Pliskova Ka.,8.0,27.0,1.61,1.62,1.67,1.60,0.0,2019
3.0_0.0_Shenzhen Open,3.0,Shenzhen,Shenzhen Open,Outdoor,Hard,Sabalenka A.,13.0,62.0,1.30,1.25,1.33,1.26,0.0,2019
4.0_0.0_Hobart International,4.0,Hobart,Hobart International,Outdoor,Hard,Kenin S.,56.0,77.0,1.44,1.46,1.50,1.44,0.0,2019
5.0_0.0_Sydney International,5.0,Sydney,Sydney International,Outdoor,Hard,Kvitova P.,8.0,15.0,1.80,1.93,1.93,1.84,0.0,2019


In [80]:
all_tournaments_wOdds.to_csv('all_tournaments_wOdds.csv')